# Introdução

As instituições financeiras estão sujeitas à perdas significativas em decorrência de inadimplência nas operações de financiamento de veículos. A elevação dos índices de inadimplemento resulta em maior taxa de rejeição de pedidos de financiamento e no aumento das taxas de juros - que encarece o produto para consumidores finais e dessestimula muitos compradores.

Diante deste cenário, torna-se cada vez mais relevante a aplicação de modelos estatísticos adequados para prever os riscos envolvidos nos contratos celebrados e permitir a concessão para clientes com menor risco de crédito.

O presente estudo procura estimar as características determinantes para inadimplência em contratos de veículos. Para isto serão utilizadas características históricas de clientes e empréstimos realizados por uma instituição financeira.



# Importação dos dados

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

import pandas_profiling
import seaborn as sns
sns.set_style("whitegrid")
sns.set(rc = {'figure.figsize':(15, 10)})

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# Carregando os dados
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

# Análise dos dados

In [ ]:
pandas_profiling.ProfileReport(train)

De acordo com o relatório apresentado pela função *ProfileReport()*, há 233.154 linhas e 41 variáveis no dataset, sendo elas 25 númericas, 6 categóricas, 6 do tipo boolean e 4 do tipo constante (*rejected*).

Observa-se que há alguns *warning* registrados para algumas variáveis do dataset que apontam alguns potenciais problemas para o modelo. São eles: alta cardinalidade, grande quantidade de valores zero na coluna, valores faltantes e um desequilíbrio na distribuição dos valores da variáveis (*y1*).

Essas observações serão tratadas ao longo da análise.

In [ ]:
# Avaliando o shape do dataset de treino e teste
train.shape, test.shape

In [ ]:
# Avaliando os típos das variáveis
train.info()

In [ ]:
# Aval
train.head().T

A partir da observação dos tipos de cada variável, algumas decisões serão executadas:
- Transformar as variáveis do tipo object em tipo inteiro ou dummy
- Transformar as datas em tipo datetime e criar colunar para contabilizar a idade da observação (em ano ou dia)
- Popular os dados faltantes com um valor "desconhecido" pela coluna como, por exemplo, o um valor "-1" em uma coluna de inteiros não-negativos


Neste trabalho, a variável dependente (*target*) será o campo **LOAN_DEFAULT**.

In [ ]:
#Analisando a quantidade de inadimplencias através da variável target LOAN_DEFAULT
train['LOAN_DEFAULT'].value_counts(normalize=True)
train['LOAN_DEFAULT'].value_counts(normalize=True).plot.bar()

Observa-se que a variável target possui um volume de valores 0 ("Inadimplente") muito maior que valores 1 ("Adimplente"). Essa será uma obervação importante para a análise do modelo.

# Tratamento dos dados

Nesta fase do trabalho os dados serão tratados de forma a permitir a aplicação de algoritmos de machine learning. 
* serão criadas novas colunas no dataframe a partir de colunas já existentes (feature engineering);
* dummerização de variáveis; 

In [ ]:
# Criação de cópia do dataframe de treino
train_copy = train.copy()

In [ ]:
train.shape, train_copy.shape

In [ ]:
# Os dataframes de teste e treino serão unidos afim de facilitar o processo de transformação e de feature engineering
train = train.append(test)

In [ ]:
train.shape

In [ ]:
#Verificando como estão os dados da coluna AVERAGE_ACCT_AGE
train['AVERAGE_ACCT_AGE'].value_counts(normalize=True)



In [ ]:
# Vamos criar nova coluna (AVERAGE_ACCT_AGE_EM_MESES), que vai contabilizar a quantidade de meses que estão dispostas na coluna AVERAGE_ACCT_AGE
train['AVERAGE_ACCT_AGE_EM_MESES'] = train['AVERAGE_ACCT_AGE'].str.split('y').str.get(0).astype(int)*12 + train['AVERAGE_ACCT_AGE'].str.split(' ').str.get(1).str.split('m').str.get(0).astype(int)

In [ ]:
train.head().T

In [ ]:
#Verificando como estão os dados da coluna CREDIT_HISTORY_LENGTH
train['CREDIT_HISTORY_LENGTH'].value_counts(normalize=True)

In [ ]:
# Vamos criar nova coluna (CREDIT_HISTORY_LENGTH_EM_MESES), que vai contabilizar a quantidade de meses que estão dispostas na coluna CREDIT_HISTORY_LENGTH
train['CREDIT_HISTORY_LENGTH_EM_MESES'] = train['CREDIT_HISTORY_LENGTH'].str.split('y').str.get(0).astype(int)*12 + train['CREDIT_HISTORY_LENGTH'].str.split(' ').str.get(1).str.split('m').str.get(0).astype(int)


In [ ]:
train.head().T

In [ ]:
# Verifica-se que a data de nascimento está na coluna DATE_OF_BIRTH
# Iremos transformar a data de nascimento em idade
train['DATE_OF_BIRTH'] = train['DATE_OF_BIRTH'].astype('datetime64[ns]')
now = pd.Timestamp('now')
train['DATE_OF_BIRTH'] = pd.to_datetime(train['DATE_OF_BIRTH'], format='%m%d%y')
train['DATE_OF_BIRTH'] = train['DATE_OF_BIRTH'].where(train['DATE_OF_BIRTH'] < now, train['DATE_OF_BIRTH'] -  np.timedelta64(100, 'Y'))
train['Idade'] = (now - train['DATE_OF_BIRTH']).astype('<m8[Y]').astype(int)

In [ ]:
# Verificando a nova coluna Idade
train['Idade']

In [ ]:
# DISBURSAL_DATE - data do desembolso
# Esta coluna será tranformada em dias afim de verificar se o tempo decorrido a partir da concessão do crédito é
# determinante do inadimplemento dos clientes

train['DISBURSAL_DATE'] = train['DISBURSAL_DATE'].astype('datetime64[ns]')
now = pd.Timestamp('now')
train['DISBURSAL_DATE'] = pd.to_datetime(train['DISBURSAL_DATE'], format='%m%d%y')
train['DISBURSAL_DATE'] = train['DISBURSAL_DATE'].where(train['DISBURSAL_DATE'] < now, train['DISBURSAL_DATE'] -  np.timedelta64(100, 'Y'))
train['DIAS_DESEMBOLSO'] = (now - train['DISBURSAL_DATE']).dt.days

In [ ]:
# verificando a quantidade de dias de desembolso
train['DIAS_DESEMBOLSO']

In [ ]:
#Verificando como estão os dados da coluna EMPLOYMENT_TYPE
train['EMPLOYMENT_TYPE'].value_counts()

In [ ]:
# Categorizando os dados, inclusive os valores nulos
le_employment_type = LabelEncoder()
train['EMPLOYMENT_TYPE'] = le_employment_type.fit_transform(list(train['EMPLOYMENT_TYPE']))
train['EMPLOYMENT_TYPE'].value_counts(normalize=True)

In [ ]:
# Verificando a quantidade de valores nulos em todo a base
train.isnull().sum()

In [ ]:
#Verificando como estão os dados da coluna PERFORM_CNS_SCORE_DESCRIPTION
train['PERFORM_CNS_SCORE_DESCRIPTION'].value_counts()

In [ ]:
#transformando em dummies
train = pd.get_dummies(train, columns=['PERFORM_CNS_SCORE_DESCRIPTION'])

In [ ]:
train.info()

# Iniciando o treinamento do modelo

De forma preliminar ao treinamento, os dados já tratados serão dividos novamente em treino e teste. Além disso, uma base de validação, para avaliar o modelo, também será gerada. 

In [ ]:
#dividindo os data sets
test = train[train['LOAN_DEFAULT'].isnull()]
train = train[~train['LOAN_DEFAULT'].isnull()]

In [ ]:
# Criando uma base de validação
train, validation = train_test_split(train, test_size = 0.30, random_state = 42)

In [ ]:
# Verificando as dimensões dos dataset
train.shape, validation.shape, test.shape

In [ ]:
# selecionar as colunas para uso no treinamento e validação

# lista das colunas não usadas
removed_cols = ['LOAN_DEFAULT','UNIQUEID', 'AVERAGE_ACCT_AGE','CREDIT_HISTORY_LENGTH','DISBURSAL_DATE','DATE_OF_BIRTH']

# lista das features (colunas usadas no modelo)
feats = [c for c in train.columns if c not in removed_cols]

In [ ]:
# Definição dos hiperparâmetros dos modelos
n_arvores = 300
oob_score = True
n_jobs = -1
n_cortes = 5
max_profundidade = 7
random_state = 42
learning_rate = 0.1

In [ ]:
# Definição do modelo de arvoré
rf = RandomForestClassifier(n_estimators = n_arvores,
                            oob_score = oob_score,
                            n_jobs = n_jobs,
                            min_samples_split = n_cortes,
                            max_depth = max_profundidade,
                            random_state = random_state)

In [ ]:
# Realização do treino do modelo
rf.fit(train[feats], train['LOAN_DEFAULT'])

In [ ]:
# Exibindo o score do out-of-bag do modelo
rf.oob_score_

In [ ]:
# Realizando as predições no modelo criado a partir do dataset de validação
predictions = rf.predict(validation[feats])

In [ ]:
# Exibindo o score da validação
accuracy_score(validation['LOAN_DEFAULT'], predictions)

In [ ]:
# Método para gerar um gráfico da matrix de confusão
# Fonte: https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

def plot_confusion_matrix(y_true, y_pred, 
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    #classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           #xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
# Plotando a matriz de confusão com valores absolutos para 
plot_confusion_matrix(validation['LOAN_DEFAULT'], predictions, 
                      title='Matriz de Confusão [Valores Absolutos], Modelo RF')

# Plotando a matriz de confusão com valores normalizados 
plot_confusion_matrix(validation['LOAN_DEFAULT'], predictions, normalize=True,
                      title='Matriz de Confusão [Valores Normalizados], Modelo RF')

In [ ]:
# Definição do modelo GCM
gbm = GradientBoostingClassifier(n_estimators = n_arvores, learning_rate = learning_rate, max_depth = max_profundidade, random_state = random_state)

In [ ]:
# Realização do treino do modelo
gbm.fit(train[feats], train['LOAN_DEFAULT'])

In [ ]:
# Realizando as predições no modelo criado a partir do dataset de validação
predictions = gbm.predict(validation[feats])

In [ ]:
# Exibindo o score da validação
accuracy_score(validation['LOAN_DEFAULT'], predictions)

In [ ]:
# Plotando a matriz de confusão com valores absolutos para 
plot_confusion_matrix(validation['LOAN_DEFAULT'], predictions, 
                      title='Matriz de Confusão [Valores Absolutos], Modelo GBM')

# Plotando a matriz de confusão com valores normalizados 
plot_confusion_matrix(validation['LOAN_DEFAULT'], predictions, normalize=True,
                      title='Matriz de Confusão [Valores Normalizados], Modelo GBM')

In [ ]:
# Definição do modelo XGBoost
xgb = XGBClassifier(n_estimators = n_arvores, learning_rate = learning_rate, random_state = random_state)

In [ ]:
# Realização do treino do modelo
xgb.fit(train[feats], train['LOAN_DEFAULT'])

In [ ]:
# Realizando as predições no modelo criado a partir do dataset de validação
predictions = xgb.predict(validation[feats])

In [ ]:
# Exibindo o score da validação
accuracy_score(validation['LOAN_DEFAULT'], predictions)

In [ ]:
# Plotando a matriz de confusão com valores absolutos para 
plot_confusion_matrix(validation['LOAN_DEFAULT'], predictions, 
                      title='Matriz de Confusão [Valores Absolutos], Modelo XGB')

# Plotando a matriz de confusão com valores normalizados 
plot_confusion_matrix(validation['LOAN_DEFAULT'], predictions, normalize=True,
                      title='Matriz de Confusão [Valores Normalizados], Modelo XGB')

In [ ]:
# Função para criação de um DF de features importantes para o modelo
def create_df_feature_importance(column_names, importances):
    df = pd.DataFrame({'feature': column_names,
                       'feature_importance': importances}) \
           .sort_values('feature_importance', ascending = False) \
           .reset_index(drop = True)
    return df

# Função para plotar o features importance do modelo
def plot_feature_importance(imp_df, title):
    imp_df.columns = ['feature', 'feature_importance']
    sns.barplot(x = 'feature_importance', y = 'feature', data = imp_df, orient = 'h', color = 'royalblue') \
       .set_title(title, fontsize = 20)

In [ ]:
# Criando o DF das principais features do modelo de RandomFlorest
base_imp = create_df_feature_importance(feats, rf.feature_importances_)

# Imprimindo as principais features utilizadas no modelo RandonFlorest
plot_feature_importance(base_imp, 'Feature Importances')

In [ ]:
# Criando o DF das principais features do modelo GBM
base_imp = create_df_feature_importance(feats, gbm.feature_importances_)

# Imprimindo as principais features utilizadas no modelo GBM
plot_feature_importance(base_imp, 'Feature Importances')

In [ ]:
# Criando o DF das principais features do modelo XGB
base_imp = create_df_feature_importance(feats, xgb.feature_importances_)

# Imprimindo as principais features utilizadas no modelo XGB
plot_feature_importance(base_imp, 'Feature Importances')

Nota-se que os modelos têm acurácia muito próximas, embora o modelo ??? tenha tido a maior acurácia dentre os três. Outra informação importante está relacionada aos falsos positivos gerados a partir do modelo. Acredita-se que a principal causa dessa anomalida seja a proporção desbalanceada dos valores presentes na variável target. Como há um percentual maior de inadimplentes, logo o modelo tende a taxar a maioria dos novos valores como inadimplentes também. Há diversas soluções para esse problema, a principal delas é normalizar a base de dados de modo que a proporção entre as classes seja o mais igualitário possível.

# Proposta de aplicação do problema ao negócio

Em relação aos dados obtidos pela aplicação do modelo de Random Forest, foi possível perceber que os principais fatores são:

LTV (Loan to value of the asset) 
* é um indicador usado para definir qual o percentual máximo do valor do bem que pode ser emprestado para o cliente. Conforme abaixo, vemos que em média até 74% do valor do bem é concedido para empréstimos para os clientes. Quando se observa os clientes com inadimplência, este percentual chega próximo de 77%, desta forma é recomendável que os empréstimos sejam menores em relação ao valor do veículo financiado


In [ ]:
train['LTV'].mean()
train['LTV'].groupby(train['LOAN_DEFAULT']).mean()

DISBURSED AMOUNT
* É o valor concedido de empréstimo. Aqui podemos observar que os empréstimos com inadiplência possuem, em média, valor mais altos de desembolso.

In [ ]:
train['DISBURSED_AMOUNT'].mean()
train['DISBURSED_AMOUNT'].groupby(train['LOAN_DEFAULT']).mean()

RISCO DE CRÉDITO
* Este é o risco atribuído pela instituição no momento do crédito do veículo
* Póssível perceber inadiplência maior em clientes com maior risco, isso sugere que a instituição financeira já detém expertise ao atribuir risco para os clientes que pode ser melhorada através deste modelo

In [ ]:
riscoA = train_copy[train_copy['PERFORM_CNS_SCORE_DESCRIPTION'] == 'A-Very Low Risk']
riscoA['LOAN_DEFAULT'].value_counts(normalize=True)

riscoE = train_copy[train_copy['PERFORM_CNS_SCORE_DESCRIPTION'] == 'E-Low Risk']
riscoE['LOAN_DEFAULT'].value_counts(normalize=True)

riscoH = train_copy[train_copy['PERFORM_CNS_SCORE_DESCRIPTION'] == 'H-Medium Risk']
riscoH['LOAN_DEFAULT'].value_counts(normalize=True)

riscoL = train_copy[train_copy['PERFORM_CNS_SCORE_DESCRIPTION'] == 'L-Very High Risk']
riscoL['LOAN_DEFAULT'].value_counts(normalize=True)